# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/yagilb/lms-hf',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/nesaorg/benchmark_v0',
 '/tencent/HunyuanVideo',
 '/black-forest-labs/FLUX.1-dev',
 '/Qwen/QwQ-32B-Preview',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/ginigen/Canvas',
 '/spaces/ginipick/SORA-3D',
 '/spaces/fantaxy/Sound-AI-SFX',
 '/spaces/ginipick/FLUXllama',
 '/spaces',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/CohereForAI/Global-MMLU',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/foursquare/fsq-os-places',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',
 '/d

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Use models from the HF Hub in LM Studio
Use Ollama with GGUF Models from the HF Hub
AI Tools are now available in HuggingChat
The AI community building the future.
The platform where the machine learning community collaborates 

In [22]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nUse models from the HF Hub in LM Studio\nUse Ollama with GGUF Models from the HF Hub\nAI Tools are now available in HuggingChat\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmeta-llama/Llama-3.3-70B-Instruct\nUpdated\nabout 11 hours ago\n•\n47.4k\n•\n821\nnesaorg/benchmark_v0\nUpdated\n6 days ago\n•\n159M\n•\n640\ntencent/HunyuanVideo\nUpdated\n4 days ago\n•\n2.54k\n•\n891\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16\n•\n1.37M\n•\n7.14k\nQwen/QwQ-32B-Preview\nUpdated\n1

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


```markdown
# Hugging Face Brochure

## Who We Are
Hugging Face is an innovative leader in the AI community, dedicated to building cutting-edge machine learning tools and fostering collaboration among developers and researchers. Our mission is to **democratize good machine learning, one commit at a time**.

## Our Platform
We provide a collaborative platform where users can:
- **Browse 400k+ models** and **100k+ datasets** that are constantly being updated.
- **Create and share Applications** including scalable 3D generations and SFX generators.
- **Access AI Tools** through HuggingChat for enhanced productivity.

## Our Community
More than **50,000 organizations** harness the power of Hugging Face, including industry giants like **Google, Microsoft, Amazon Web Services, and Intel**. Our community is passionate about pushing the boundaries of AI and machine learning, revealing the potential of open-source collaboration.

## Company Culture
At Hugging Face, we celebrate a **dynamic and inclusive workspace**, with a team of **over 225 members** who share a commitment to innovation and community-driven development. We strive to create an environment that champions collaboration, creativity, and continuous learning. 

Are you passionate about AI and looking for a place where your contributions make a difference? Join us on our mission!

## Career Opportunities
Hugging Face is always looking to expand our talented team. We offer various career paths for individuals who are enthusiastic about machine learning and artificial intelligence. Explore open positions and build your career with us!

## Get Involved
Whether you're a seasoned data scientist or a newcomer to the field, we encourage you to:
- **Join our community**: Engage in discussions and share your projects.
- **Contribute to our open-source models**: We're constantly evolving and every contribution counts.

## Learn More
For more details on our offerings, pricing, and how to get started, visit our website at [Hugging Face](https://huggingface.co).

Let's shape the future of AI together!
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}



# Hugging Face Brochure

## Who We Are
**Hugging Face** is an innovative AI community dedicated to building the future of artificial intelligence and machine learning. We provide a collaborative platform where machine learning enthusiasts, researchers, and professionals can create, discover, and share models, datasets, and applications.

## Our Mission
Our mission is to democratize access to machine learning tools and resources, enabling everyone to harness the power of AI. With over 50,000 organizations relying on our platform, we are at the forefront of shaping the future of technology.

## What We Offer
- **Models**: Access to over 400,000 models ranging from language processing to image generation.
- **Datasets**: A comprehensive library of more than 100,000 datasets for various tasks in machine learning.
- **Spaces**: Create and collaborate in applications designed for real-time interaction and innovation.
- **Enterprise Solutions**: Scalable compute solutions with enterprise-grade security and support, starting at just $20/user/month.

## Our Customers
Hugging Face serves a diverse clientele, including major corporations and non-profits:
- **AI at Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Grammarly**

## Company Culture
At Hugging Face, we believe in community, innovation, and transparency. Our team consists of passionate individuals dedicated to collaboration. We encourage open source contributions, and our culture revolves around continuous learning and building together.

## Career Opportunities
Join our team of 225+ experts, creatives, and thinkers. We seek individuals who are eager to help democratize machine learning and are looking for potential candidates across various roles.

If you’re passionate about ML, we invite you to explore our career opportunities! 

- **[Explore Careers at Hugging Face](https://huggingface.co/jobs)**

## Connect with Us
Stay updated on our latest advancements and community initiatives:
- **GitHub**: [huggingface](https://github.com/huggingface)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face](https://linkedin.com/company/huggingface)
- **Discord**: [Join our Community](https://huggingface.co/discord)

## Join the Movement
We are just getting started! If you’re enthusiastic about AI and want to be part of an exciting journey, **sign up** today and help us build machine learning tools that impact the world!

**[Sign Up Now](https://huggingface.co/signup)**



In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co'}]}


# 📚 Welcome to Hugging Face! 🤗

## Who Are We?
At **Hugging Face**, we are the friendly neighborhood superheroes of **AI and Machine Learning** (or just your everyday nerds with a passion)! Our mission? To democratize good machine learning, one commit at a time! Join the **AI community building the future**. (No capes required! 🦸‍♂️)

---

## What We Offer
- **💾 Models:** Browse through **400k+** models that will make you question how in the world machines got this smart!
- **📊 Datasets:** Access **100k+** datasets, because, after all, data is the new oil! (And way easier to spill!)
- **💻 Spaces:** Create and share applications like you’re hosting a bake sale - only this time, the cookies are made of complex algorithms!

---

## Our Culture 🌍
- **Collaboration:** At Hugging Face, we strongly believe in teamwork (even if it means arguing about which cat video to watch next during breaks! 🐱). 
- **Innovation:** With platforms running models that are trending (who doesn't want a little celebrity moment?), we encourage creativity that shakes things up!
- **Support:** Whether you need help solving a technical glitch or finding the best coffee in the office, we’ve got your back—like a trusty AI assistant!

---

## Our Customers 🛍️
More than **50,000 organizations** are utilizing our advanced AI tools. From **AI at Meta** to the crème de la crème of tech like **Amazon**, **Google**, and even **Grammarly**—everyone’s in on it! If you're not using us yet, don’t worry; there’s always room for one more. 

---

## Careers With Us 🚀
Ready to jump in? We're on the lookout for more talent! Our team consists of 225+ amazing individuals (with a collective IQ that could probably power a spaceship 🌌). 

### Why Join Us?
- **Work-life balance:** We understand the importance of taking breaks—especially for coffee refills and stretching those coding fingers!
- **Growth opportunities:** Learn and grow faster than a piece of stale bread in a microwave! 
- **Diversity:** We welcome everyone because we believe every mind can help change the world (or at least help us get beyond level 5 on our game night).

---

## Join the Fun! 🎉
Curious? Excited? Ready to help build the future? Head over to **[Hugging Face](https://huggingface.co)** and become part of our journey—one algorithm at a time!

> P.S. Optional: Bring your favorite meme to the interview—you might just snag bonus points! 🏆

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>